# Baseline - Most popular
Most popular books are a combination of vote count and the weighted rating

In [29]:
from bibrec.server.Utils import assign_popular_based_score
import pandas as pd
import numpy as np
import bibrec.server.evaluation as eval
from collections import defaultdict
import bibrec.server.Utils as Utils
import importlib
from sklearn.model_selection import train_test_split

importlib.reload(eval)
importlib.reload(Utils)

books, users, ratings = Utils.get_normalized_data()

/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=";", encoding="latin-1")
/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:98: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


Normalized Ratings Count: 383962


In [30]:
train, test = train_test_split(ratings, test_size=0.25)

In [31]:
print("Total Ratings Count:", len(ratings))
print("Train Data Ratings Count:", len(train))
print("Test Data Ratings Count:", len(test))

Total Ratings Count: 383962
Train Data Ratings Count: 287971
Test Data Ratings Count: 95991


In [32]:
book_ratings_dict = {
    'user_id': train['user_id'].values,
    'isbn13': train['isbn13'].values,
    'book_rating': train['book_rating'].values
}

book_rating_df = train[['user_id', 'isbn13', 'book_rating']]

In [33]:
train["isbn13"] = train["isbn13"].astype("int")
books["isbn13"] = books["isbn13"].astype("int")
popular_books = assign_popular_based_score(train, books, "user_id", "isbn13", "book_rating")
popular_books = popular_books.sort_values("vote_count", ascending=False)
popular_books = popular_books[:200]
popular_books = popular_books.sort_values("weighted_rating", ascending=False)

print("Most popular books are:")
print(popular_books[:20])

Most popular books are:
              isbn13  vote_count  avg_rating  weighted_rating
16320  9780439139595         108    9.194444         9.164239
928     978034533976          72    9.111111         9.068464
1548    978043913961          81    9.049383         9.012847
23460  9780590353427         236    9.004237         8.991875
16316  9780439136358         112    9.017857         8.991810
16317  9780439136365          92    9.021739         8.990067
16277  9780439064866          99    9.010101         8.980855
16372  9780439358064         144    8.951389         8.931961
17652  9780446310789         168    8.892857         8.876861
17174  9780440498056          68    8.897059         8.858090
23459  9780590353403         100    8.880000         8.853591
9808   9780345339683         119    8.823529         8.802201
9809   9780345339706         101    8.801980         8.777343
16278  9780439064873         145    8.731034         8.714737
14110  9780385484510         149    8.718121  

In [34]:
pop_dict = {
    'item_id': popular_books['isbn13'].values,
    'est_r': popular_books['weighted_rating'].values,
}

pop_df = pd.DataFrame.from_dict(pop_dict)

ratings_dict = {
    'user_id': test['user_id'].values,
    'item_id': test['isbn13'].values,
    'rating': test['book_rating'].values
}

ratings_df = pd.DataFrame.from_dict(ratings_dict)

In [35]:
test_uids = ratings_df["user_id"].unique()

print("User Count:", len(test_uids))

User Count: 29820


In [36]:
def def_value():
    return "Not Present"


top_n = defaultdict(def_value)
for uid in test_uids:
    top_n[uid] = pop_df

In [37]:
import time

start_time = time.time()
avg_precision = eval.get_avg_precision(ratings_df, top_n, k=50)

print("Average Precision:", avg_precision)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Precision: 0.0008469757232790124
--- Calculation time: 55.491865158081055 seconds ---


In [38]:
avg_recall = eval.get_avg_recall(ratings_df, top_n, k=50)

print("Average Recall:", avg_recall)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Recall: 0.013461825054893737
--- Calculation time: 114.11931920051575 seconds ---
